# Face Recognition for the Happy House

Trong notebook này, chúng ta sẽ làm quen với mô hình Facenet cho bài toán Face Recognition
Các bài toán nhận dạng khuôn mặt thường chia thành hai loại:

- **Face Verification** (**Xác thực**) - "Đây có phải người X không?". Một vài ứng dụng có thể kể đến là xác nhận dạng ảnh chân dung trên passport hoặc các kệ thống đăng nhập bằng ảnh mặt người dùng. Đây là bài toáng so sánh 1:1.
- **Face Recognition** (**Nhận diện**) - "Người này là ai?". Ví dụ có thể kể đến là hệ thống checkin nhân viên tại công ty. Đây là bài toán so sánh 1:1K.

Mô hình Facenet sẽ học ra một mạng neuron nhằm mã hóa một bức ảnh thành 1 vector 128 chiều (có 128 số). Bằng cách so sánh khoảng cách giữa hai vector như thế, ta có thể xác định xem hai bức ảnh có thuộc về cùng một người hay không.

**Trong assignment này, bạn sẽ:**
- Lập trình hàm triplet loss
- Sử dụng mô hình pretrained để mã hóa khuôn mặt thành các vector 128 chiều
- Sử dụng các vector này để thực hiện face recognition và face verification

Trong bài tập, ta sẽ sử dụng mô hình một pre-trained sử dụng convention "channels first", trái ngược với convention "channels last". Cụ thể, một một batch trong training sẽ có shape là  $(m, n_C, n_H, n_W)$ (số ảnh trong batch, số kênh, chiều cao, chiều rộng), thay vì $(m, n_H, n_W, n_C)$.

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform
from keras.engine.topology import Layer
from keras import backend as K
K.set_image_data_format('channels_first')
import cv2
import os
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf
from fr_utils import *
from inception_blocks_v2 import *

%matplotlib inline
%load_ext autoreload
%autoreload 2

np.set_printoptions(threshold=np.nan)

## 0 - Face Verification đơn giản

Trong bài toán Face Verification, bạn được đưa hai bức ảnh và nhiệm vụ của bạn là xác định xem hai bức ảnh có thuộc về cùng một người hay không. Cách đơn giản nhất là so sánh từng cặp pixel của hai hình ảnh với nhau. Nếu sự sai khác giữa hai hình ảnh thấp hơn một ngưỡng đã được cho từ trước, đây có thể là cùng một người.

<img src="images/pixel_comparison.png" style="width:380px;height:150px;">
<caption><center> <u> <font color='purple'> **Figure 1** </u></center></caption>

Đương nhiên, cách làm này sẽ cho kết quả rất kém, vì giá trị pixel ảnh sẽ thay đổi rất nhiều do cách yếu tố như độ sáng, góc chụp khuôn mặt, thậm chí sự thay đổi tư thể của đầu,...

Ta thấy thay vì sử dụng ảnh thô, chúng ta có thể học cách mã hóa thông tin ảnh để việc đo đạc sự sai khác này được chính xác hơn

## 1 - Mã hóa ảnh khuôn mặt thành vector 128 chiều

### 1.1 - Sử dụng ConvNet  để tính toán vector mã hóa

Mô hình Facenet cần một lượng lớn data và nhiều thời gian để huấn luyện. Vì vậy, ta sẽ chỉ sử dụng mô hình đã được train hẵn bởi người khác. Kiến trúc mạng sử dụng trong bài được thiết kế theo mô hình Inception trong bài báo [Szegedy *et al.*](https://arxiv.org/abs/1409.4842). Chúng ta đã có sẵn phần lập trình Inception network trong file `inception_blocks.py`.

Những điểm quan trọng bài cần nhớ là:
- Mạng sử dụng đầu vào là ảnh RGB có kích thước 96*96. Cụ thể, mỗi training batch sẽ có kích thước $(m, n_C, n_H, n_W) = (m, 3, 96, 96)$ (m* là số mẫu trong một batch)
- Đầu ra của mạng là một ma trận có kích cỡ $(m, 128)$, với mỗi hàng đại diện cho một ảnh đẽ được mã hóa thành vector 128 chiều

Chạy Cell phía dưới để tạo ra model cho ảnh khuôn mặt

In [ ]:
FRmodel = faceRecoModel(input_shape=(3, 96, 96))

In [ ]:
print("Total Params:", FRmodel.count_params())

** Expected Output **
<table>
<center>
Total Params: 3743280
</center>
</table>


Bằng cách sử dụng một tầng fully connected với 128 neuron làm layer cuối cùng, mô hình trả về đầu ra là một vector có size 128. Sau đó, ta dùng những vector này để tính sự sai khác giữa các khuôn mặt như sau:

<img src="images/distance_kiank.png" style="width:680px;height:250px;">
<caption><center> <u> <font color='purple'> **Figure 2**: <br> </u> <font color='purple'> Tính sự sai khác giữa hai ảnh và lấy ngưỡng là cách để xác nhận xem hai bức ảnh có thuộc về cùng một người hay không</center></caption>

Vì thế, một mô hình mã hóa là tốt nếu:
- Hai bức ảnh của cùng một người có giá trị mã hóa khá giống nhau
- Hai bức ảnh của hai người khác nhau có giá trị mã hóa khác xa nhau
    
Ta ý tưởng này được thể hiện bằng hàm triplet loss, với hai giá trị mã hóa của cùng một người được "đẩy" lại gần nhau (Anchor và Positive), trong khi "kéo" giá trị mã hóa ảnh của hai người khác nhau ra xa nhau (Anchor, Negative).
 
<img src="images/triplet_comparison.png" style="width:280px;height:150px;">
<br>
<caption><center> <u> <font color='purple'> **Figure 3**: <br> </u> <font color='purple'> Trong phần tiếp theo, ta sẽ gọi các ảnh từ trác qua phải lần lượt như sau: Anchor (A), Positive (P), Negative (N)  </center></caption>



### 1.2 - Hàm Triplet Loss

Cho một ảnh $x$, ta ký hiệu giá trị mã hóa của nó là $f(x)$, với $f$ là hàm số được tính toán bởi mạng neuron.

<img src="images/f_x.png" style="width:380px;height:150px;">

<!--
Ta cũng sẽ cho thêm một bước chuẩn hóa tại phía cuối của mô hình để $\mid \mid f(x) \mid \mid_2 = 1$ (vector mã hóa được đưa về khoảng giá trị (0,1)).
!-->

Mỗi bản ghi dùng cho huấn luyện sẽ là bộ ba ảnh $(A, P, N)$:  

- A là ảnh "Anchor"--một bức ảnh của một người. 
- P là ảnh "Positive"--một bức ảnh của cùng người đó.
- N là ảnh "Negative"--một bức ảnh của một người khác.

Những bộ ba này được chọn ra từ tập data huấn luyện.Ta sẽ viết $(A^{(i)}, P^{(i)}, N^{(i)})$ để ký hiệu bản ghi huấn luyện thứ $i$. 

Bạn sẽ muốn chắc chắn rằng một hình ảnh $A^{(i)}$ sẽ ở gần với ảnh Positive $P^{(i)}$ hơn là ảnh Negative $N^{(i)}$) bằng ít nhất một lượng $\alpha$:

$$\mid \mid f(A^{(i)}) - f(P^{(i)}) \mid \mid_2^2 + \alpha < \mid \mid f(A^{(i)}) - f(N^{(i)}) \mid \mid_2^2$$

Vì thế công việc của bạn là cực tiểu hóa hàm "triplet cost" dưới đây:

$$\mathcal{J} = \sum^{N}_{i=1} \large[ \small \underbrace{\mid \mid f(A^{(i)}) - f(P^{(i)}) \mid \mid_2^2}_\text{(1)} - \underbrace{\mid \mid f(A^{(i)}) - f(N^{(i)}) \mid \mid_2^2}_\text{(2)} + \alpha \large ] \small_+ \tag{3}$$

Ở đây, chúng ta sử dụng biểu tượng "$[z]_+$" làm ký hiệu cho $max(z,0)$.  

Ghi chú:
- Vế (1) là bình phương khoảng cách giữa Anchor "A" và Positive "P" trong bộ ba ảnh, giá trị này cần càng nhỏ càng tốt. 
- Vế (2) là bình phương khoảng cách giữa Anchor "A" và Negative "N" trong bộ ba ảnh, giá trị này cần tương đối lớn. 
- $\alpha$ là ngưỡng. Đây là siêu tham số ta tự chọn. Ta sẽ dùng $\alpha = 0.2$. 

Đa phần các ứng dụng lập trình có triplet loss sẽ có thêm một phần chuẩn hóa vector về khoảng (0,1) (i.e., $\mid \mid f(img)\mid \mid_2$=1); ở đây chúng ta chưa cần quan tâm.

**Bài tập**: Lập trình hàm triplet loss được đinh nghĩa như trên. Dưới đây là 4 bước:
1. Tính khoảng cách giữa giá trị mã hóa của Anchor và Positive: $\mid \mid f(A^{(i)}) - f(P^{(i)}) \mid \mid_2^2$
2. Tính khoảng cách giữa giá trị mã hóa của Anchor và Negative: $\mid \mid f(A^{(i)}) - f(N^{(i)}) \mid \mid_2^2$
3. Tính công thức sau cho mỗi training example: $ \mid \mid f(A^{(i)}) - f(P^{(i)}) \mid - \mid \mid f(A^{(i)}) - f(N^{(i)}) \mid \mid_2^2 + \alpha$
4. Tính hàm đầy đủ bằng cách lấy max với 0 và tính tổng trên tất cả các bản ghi:
$$\mathcal{J} = \sum^{N}_{i=1} \large[ \small \mid \mid f(A^{(i)}) - f(P^{(i)}) \mid \mid_2^2 - \mid \mid f(A^{(i)}) - f(N^{(i)}) \mid \mid_2^2+ \alpha \large ] \small_+ \tag{3}$$

Những hàm hữu ích: `tf.reduce_sum()`, `tf.square()`, `tf.subtract()`, `tf.add()`, `tf.reduce_mean`, `tf.maximum()`.

In [ ]:
# GRADED FUNCTION: triplet_loss

def triplet_loss(y_true, y_pred, alpha = 0.2):
    """
    Implementation of the triplet loss as defined by formula (3)
    
    Arguments:
    y_true -- true labels, required when you define a loss in Keras, you don't need it in this function.
    y_pred -- python list containing three objects:
            anchor -- the encodings for the anchor images, of shape (None, 128)
            positive -- the encodings for the positive images, of shape (None, 128)
            negative -- the encodings for the negative images, of shape (None, 128)
    
    Returns:
    loss -- real number, value of the loss
    """
    
    anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]
    
    ### START CODE HERE ### (≈ 4 lines)
    # Step 1: Compute the (encoding) distance between the anchor and the positive
    pos_dist = None
    # Step 2: Compute the (encoding) distance between the anchor and the negative
    neg_dist = None
    # Step 3: subtract the two previous distances and add alpha.
    basic_loss = None
    # Step 4: Take the maximum of basic_loss and 0.0. Sum over the training examples.
    loss = None
    ### END CODE HERE ###
    
    return loss

In [ ]:
with tf.Session() as test:
    tf.set_random_seed(1)
    y_true = (None, None, None)
    y_pred = (tf.random_normal([3, 128], mean=6, stddev=0.1, seed = 1),
              tf.random_normal([3, 128], mean=1, stddev=1, seed = 1),
              tf.random_normal([3, 128], mean=3, stddev=4, seed = 1))
    loss = triplet_loss(y_true, y_pred)
    
    print("loss = " + str(loss.eval()))

**Expected Output**:

<table>
    <tr>
        <td>
            **loss**
        </td>
        <td>
           350.026
        </td>
    </tr>

</table>

## 2 - Load mô hình huấn luyện từ trước

FaceNet được huấn luyện với hàm triplet loss. Nhưng vì việc training yêu cầu rất nhiều thời gian và data, ta sẽ không huấn luyện lại từ đầu. Thay vào đó, tã sẽ sử dụng mô hình được train từ trước. Chạy cell phía dưới để thực hiện việc load mô hình, việc này có thể mất vài phút. 

In [ ]:
FRmodel.compile(optimizer = 'adam', loss = triplet_loss, metrics = ['accuracy'])
load_weights_from_FaceNet(FRmodel)

Dưới đây là một vài ví dụ về khoảng cách giữa các ảnh:

<img src="images/distance_matrix.png" style="width:380px;height:200px;">
<br>
<caption><center> <u> <font color='purple'> **Figure 4**:</u> <br>  <font color='purple'> Example of distance outputs between three individuals' encodings</center></caption>

Bây giờ ta sẽ sử dụng mô hình để nhận diện khuôn mặt! 

## 3 - Sử dụng mô hình

Đâu tiên, ta sẽ xây dựng một hệ thống **Face verification**. 

### 3.1 - Face Verification

Trước tiên, ta xây dựng một database cho những người cần xác thực, để tạo ra vector mã hóa ta dùng hàm`img_to_encoding(image_path, model)` để thực hiện quá trình inference với model. 

Chạy đoạn code dưới đây để tạo ra dictionary với mỗi key là tên người nhận diện, value là một vector mã hóa có 128 chiều.

In [ ]:
database = {}
database["danielle"] = img_to_encoding("images/danielle.png", FRmodel)
database["younes"] = img_to_encoding("images/younes.jpg", FRmodel)
database["tian"] = img_to_encoding("images/tian.jpg", FRmodel)
database["andrew"] = img_to_encoding("images/andrew.jpg", FRmodel)
database["kian"] = img_to_encoding("images/kian.jpg", FRmodel)
database["dan"] = img_to_encoding("images/dan.jpg", FRmodel)
database["sebastiano"] = img_to_encoding("images/sebastiano.jpg", FRmodel)
database["bertrand"] = img_to_encoding("images/bertrand.jpg", FRmodel)
database["kevin"] = img_to_encoding("images/kevin.jpg", FRmodel)
database["felix"] = img_to_encoding("images/felix.jpg", FRmodel)
database["benoit"] = img_to_encoding("images/benoit.jpg", FRmodel)
database["arnaud"] = img_to_encoding("images/arnaud.jpg", FRmodel)

**Exercise**: Lập trình hàm verify() để kiểm tra một ảnh trong thư mục (`image_path`) có đúng là một người gọi là nào đó (gọi là "identity") hay không. Bạn sẽ phải đi qua những bước sau:
1. Tính toán giá trị vector của mô hình từ thư mục image_path
2. Tính toán khoảng cách của vector từ bước một với giá trị mã khóa của ảnh thuộc về người có cùng tên trong database
3. Mở cửa nếu giá trị bé hơn 0.7.

Chúng ta sẽ tính khoảng cách L2 bằng hàm (np.linalg.norm). (Ghi chú: Ta so sánh khoảng L2 với ngưỡng 0.7, không phải bình phương khoảng cách L2. 

In [ ]:
# GRADED FUNCTION: verify

def verify(image_path, identity, database, model):
    """
    Function that verifies if the person on the "image_path" image is "identity".
    
    Arguments:
    image_path -- path to an image
    identity -- string, name of the person you'd like to verify the identity. Has to be a resident of the Happy house.
    database -- python dictionary mapping names of allowed people's names (strings) to their encodings (vectors).
    model -- your Inception model instance in Keras
    
    Returns:
    dist -- distance between the image_path and the image of "identity" in the database.
    door_open -- True, if the door should open. False otherwise.
    """
    
    ### START CODE HERE ###
    
    # Step 1: Compute the encoding for the image. Use img_to_encoding() see example above. (≈ 1 line)
    encoding = None
    
    # Step 2: Compute distance with identity's image (≈ 1 line)
    dist = None
    
    # Step 3: Open the door if dist < 0.7, else don't open (≈ 3 lines)
    if None:
        print("It's " + str(identity) + ", welcome home!")
        door_open = None
    else:
        print("It's not " + str(identity) + ", please go away")
        door_open = None
        
    ### END CODE HERE ###
        
    return dist, door_open

Younes là người trong ảnh dưới đây, tên anh ta có trong database:

<img src="images/camera_0.jpg" style="width:100px;height:100px;">

In [ ]:
verify("images/camera_0.jpg", "younes", database, FRmodel)

**Expected Output**:

<table>
    <tr>
        <td>
            **It's younes, welcome home!**
        </td>
        <td>
           (0.65939283, True)
        </td>
    </tr>

</table>

Benoit, người trong ảnh dưới, đã lấy trộm ID card của Kian để xác nhận, hãy chạy đoạn mã dưới để xem anh ta có được xác nhận là Kian hay không.
<img src="images/camera_2.jpg" style="width:100px;height:100px;">

In [ ]:
verify("images/camera_2.jpg", "kian", database, FRmodel)

**Expected Output**:

<table>
    <tr>
        <td>
            **It's not kian, please go away**
        </td>
        <td>
           (0.86224014, False)
        </td>
    </tr>

</table>

### 3.2 - Face Recognition

Bây giờn chúng ta sẽ xây dựng một hệ thống face recognition nhận đầu vào là ảnh của một người, đầu ra là kết quả xem người đó là ai trong database. 

**Bài tập**: Lập trình hàm `who_is_it()`. Bạn sẽ phải đi qua những bước sau:
1. Tính toán giá trị mã hóa của ảnh đầu vào từ thư mục image_path
2. Tìm giá trị mã hóa trong data base gần với lại giá trị từ bước 1 nhất. 
    - Khởi tạo giá trị `min_dist` bằng một số đủ lớn, ví dụ 100. Nó sẽ giúp bạn theo dõi được giá trị khoảng cách bé nhất.
    - Tạo một vòng lặp với tên và giá trị mã hóa trong database. Gợi ý: sử dụng `for (name, db_enc) in database.items()`.
        - Tính khoảng cách L2 "encoding" và encoding hiện tại, cho là biến dist.
        - Nếu dist < min_dist, set min_dist thành dist, và chuyển đặt biến identity = tên người có dist = min_dist hiện tại.

In [ ]:
# GRADED FUNCTION: who_is_it

def who_is_it(image_path, database, model):
    """
    Implements face recognition for the happy house by finding who is the person on the image_path image.
    
    Arguments:
    image_path -- path to an image
    database -- database containing image encodings along with the name of the person on the image
    model -- your Inception model instance in Keras
    
    Returns:
    min_dist -- the minimum distance between image_path encoding and the encodings from the database
    identity -- string, the name prediction for the person on image_path
    """
    
    ### START CODE HERE ### 
    
    ## Step 1: Compute the target "encoding" for the image. Use img_to_encoding() see example above. ## (≈ 1 line)
    encoding = None
    
    ## Step 2: Find the closest encoding ##
    
    # Initialize "min_dist" to a large value, say 100 (≈1 line)
    min_dist = None
    
    # Loop over the database dictionary's names and encodings.
    for (name, db_enc) in None:
        
        # Compute L2 distance between the target "encoding" and the current "emb" from the database. (≈ 1 line)
        dist = None

        # If this distance is less than the min_dist, then set min_dist to dist, and identity to name. (≈ 3 lines)
        if None:
            min_dist = None
            identity = None

    ### END CODE HERE ###
    
    if min_dist > 0.7:
        print("Not in the database.")
    else:
        print ("it's " + str(identity) + ", the distance is " + str(min_dist))
        
    return min_dist, identity

Người trong ảnh là Younes ("images/camera_0.jpg").

In [ ]:
who_is_it("images/camera_0.jpg", database, FRmodel)

**Expected Output**:

<table>
    <tr>
        <td>
            **it's younes, the distance is 0.659393**
        </td>
        <td>
           (0.65939283, 'younes')
        </td>
    </tr>

</table>

Bạn có thể thử với các ảnh khác trong thư mục để xem kết quả.

Một vài gợi ý để cải thiện hệ thống face recognition:
- Với mỗi người ta có nhiều ảnh mã hóa (khác nhau về góc chụp, độ sáng,...) trong database, như thế sẽ làm tăng thêm độ chính xác của mô hình.
- Cắt ảnh sao cho chỉ giữ lại phần mặt, mô hình sẽ hoạt động tốt hơn vì không phải tính toán trên các giá trị pixel thừa.

<font color='blue'>
**Key takeaway**:
  
- Bài toán Face verification là so sánh 1:1, còn face recognition là so sánh 1:K. 
- Hàm loss sử dụng để train mô hình mã hóa ảnh khuôn mặt là triplet loss.
- Cùng một giá trị mã hóa có thể dùng cho cả bài toán verification and recognition. Sau đó chúng ta đo khoảng cách giữa hai vector mã hóa của hai ảnh để xác nhận xem hai bức ảnh có thuộc về cùng một người không. 

Chúc mừng bạn đã hoàn thành khóa học!

### References:

- Florian Schroff, Dmitry Kalenichenko, James Philbin (2015). [FaceNet: A Unified Embedding for Face Recognition and Clustering](https://arxiv.org/pdf/1503.03832.pdf)
- Yaniv Taigman, Ming Yang, Marc'Aurelio Ranzato, Lior Wolf (2014). [DeepFace: Closing the gap to human-level performance in face verification](https://research.fb.com/wp-content/uploads/2016/11/deepface-closing-the-gap-to-human-level-performance-in-face-verification.pdf) 
- Mô hình pretrained được lấy từ implementation của Victor Sy Wang và được load sử dụng code của tác giả: https://github.com/iwantooxxoox/Keras-OpenFace.
- Repository chính thức của mô hình FaceNet trên github: https://github.com/davidsandberg/facenet 
- Notebook được biên soạn lại theo bản gốc của tác giả của Andrew Ng.